<a href="https://colab.research.google.com/github/Worachet-Ch/project2022/blob/main/random_NaN_by_Probability_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
import numpy as np
from sklearn.impute import SimpleImputer
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error
import time
from sklearn import neighbors
from sklearn.impute import KNNImputer
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [171]:
import pandas as pd
import os
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'dataset2.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

In [172]:
data

,No.,LabNo,Sex,AgeSYear,Enose,Overall,Body shape age,Kidney,Heart,Liver,...,071_HbA1c,CK,LD-P,AMY7,TP1,DBIL,TBIL,ALB1,GGT,homocysteine
0,1.0,NaN,2,41.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.8,79.0,231.0,59.0,7.16,0.10,0.30,4.48,11.0,10.88
1,2.0,201807163166.00,1,49.0,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.11
2,3.0,201807163167,1,44.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.6,85.0,175.0,104.0,7.43,0.15,1.26,4.70,15.0,11.94
3,4.0,201807163168,2,37.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.01
4,5.0,201807163169,2,38.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4,57.0,177.0,89.0,7.11,0.02,0.20,3.92,24.0,8.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269,NaN,V17,2,31.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2270,NaN,V19,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2271,NaN,V20,1,28.0,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2272,NaN,V21,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
data.dropna(subset=['telomere length (kb) '], inplace = True)
data.drop(['No.', 'LabNo','DHEAs (BRIA)'], axis=1,inplace = True)
data.drop(data.columns[2: 10], axis=1,inplace = True)
data.drop(data.columns[27:37], axis=1,inplace = True)
data.drop('telomere length (kb) ',1,inplace = True)

<ipython-input-173-1a7942661c33>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop('telomere length (kb) ',1,inplace = True)


In [174]:
data

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
49,2,58.0,47.2,156.0,19.4,112.0,64.0,70.0,NaN,NaN,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
56,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
85,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
103,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
107,2,86.0,49.0,154.0,20.7,129.0,79.0,83.0,NaN,NaN,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,2,25.0,47.0,165.0,17.3,104.0,63.0,89.0,NaN,NaN,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
2269,2,31.0,72.2,168.0,25.6,123.0,89.0,69.0,NaN,NaN,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
2270,2,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
2272,1,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [175]:
P = 0
p1 = 0
list_mis = []
for i in data:
  if data[i].isna().sum() == 0 :
    continue
  else:
    ms = data[i].isna().sum()
    P = (ms/1734).round(14)
    p1 = p1 + P
    print(f'ความน่าจะเป็นของ {i} = {P}')
    list_mis.append(P)
print('****************************************')
print(f'ความน่าจะเป็นทั้งหมด = {p1}')

ความน่าจะเป็นของ Weight = 0.00461361014994
ความน่าจะเป็นของ Height = 0.00461361014994
ความน่าจะเป็นของ BMI = 0.00461361014994
ความน่าจะเป็นของ Systolic = 0.00461361014994
ความน่าจะเป็นของ Diastolic = 0.00519031141869
ความน่าจะเป็นของ HeartRate = 0.02941176470588
ความน่าจะเป็นของ SMM = 0.12975778546713
ความน่าจะเป็นของ Fat Mass = 0.12975778546713
ความน่าจะเป็นของ % Body fat = 0.12975778546713
ความน่าจะเป็นของ waist to hip radio = 0.12975778546713
ความน่าจะเป็นของ abdominal circumference = 0.13033448673587
ความน่าจะเป็นของ visceral fat area = 0.12975778546713
ความน่าจะเป็นของ 001_Glucose = 0.00057670126874
ความน่าจะเป็นของ 004_Uric Acid = 0.05363321799308
ความน่าจะเป็นของ 012_ALT = 0.00057670126874
ความน่าจะเป็นของ 013_Alkaline Phos = 0.01672433679354
ความน่าจะเป็นของ 071_HbA1c = 0.09630911188005
****************************************
ความน่าจะเป็นทั้งหมด = 1.0


In [176]:
data.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [177]:
data_file_path1 = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data1 = pd.read_csv(data_file_path1) #อ่านไฟล์ csv

In [178]:
data1.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)

<ipython-input-178-0e7a2b46e4dd>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data1.drop({'Unnamed: 0','telomere length (kb) '},1,inplace = True)


In [179]:
data1.rename(columns={'Fat Mass':'FatMass','% Body fat':'Bodyfat','waist to hip radio':'waisttohip','abdominal circumference':'abdominal',
                    'visceral fat area':'visceral','004_Uric Acid':'UricAcid',''
                                'HDL-C':'HDL','013_Alkaline Phos':'Alkaline_Phos'},inplace = True)

In [180]:
data1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [181]:
list1 = []
for i in data:
  if data[i].isna().sum() == 0:
    continue
  else: 
    list1.append(i)
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [182]:
list_mis

[0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00461361014994,
 0.00519031141869,
 0.02941176470588,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.12975778546713,
 0.13033448673587,
 0.12975778546713,
 0.00057670126874,
 0.05363321799308,
 0.00057670126874,
 0.01672433679354,
 0.09630911188005]

# 10 %

## แทนค่า NaN

In [ ]:
random_column = np.random.choice(list1,int((153*17)*(10/100)),p=list_mis)
random_column = list(random_column)
random_column

In [184]:
weight = []
height = []
bmi = []
systolic = []
diastolic = []
heartrate = []
smm = []
fatmass = []
bodyfat = []
wait = []
abdominal = []
visceral = []
glucose = []
uric_acid = []
ALT = []
Alkaline = []
HbA1c = []

In [185]:
for i in random_column:
  if i == 'Weight':
    weight.append(i)  
  elif i == 'Height':
    height.append(i)
  elif i == 'BMI':
    bmi.append(i)
  elif i == 'Systolic':
    systolic.append(i)
  elif i == 'Diastolic':
    diastolic.append(i)
  elif i == 'HeartRate':
    heartrate.append(i)
  elif i == 'SMM':
    smm.append(i)
  elif i == 'Fat Mass':
    fatmass.append(i)
  elif i == '% Body fat':
    bodyfat.append(i)
  elif i == 'waist to hip radio':
    wait.append(i)
  elif i == 'abdominal circumference':
    abdominal.append(i)
  elif i == 'visceral fat area':
    visceral.append(i)
  elif i == '001_Glucose':
    glucose.append(i)
  elif i == '004_Uric Acid':
    uric_acid.append(i)
  elif i == '012_ALT':
    ALT.append(i)
  elif i == '013_Alkaline Phos':
    Alkaline.append(i)
  elif i == '071_HbA1c':
    HbA1c.append(i)

In [186]:
list2 = [len(weight),len(height),len(bmi),len(systolic),len(diastolic),len(heartrate),len(smm),
         len(fatmass),len(bodyfat),len(wait),len(abdominal),len(visceral),len(glucose),len(uric_acid),
         len(ALT),len(Alkaline),len(HbA1c)]
list3 = []
list4 = []

for i in range(153):
  m = 1/153
  list3.append(i)
  list4.append(m)

In [187]:
list2

[1, 1, 0, 0, 0, 9, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24]

In [188]:
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [ ]:
list_071_HbA1c

In [190]:
prob_Weight

0.0

In [191]:
dt10 = data1.copy()
dt10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [192]:
for i,j in zip(list1,list2):
  dt10[i].ravel()[np.random.choice(list3,j,p= list4, replace=False)] = np.nan

In [193]:
dt10.isna().sum()

Sex                  0
AgeSYear             0
Weight               1
Height               1
BMI                  0
Systolic             0
Diastolic            0
HeartRate            9
SMM                 23
FatMass              0
Bodyfat              0
waisttohip           0
abdominal            0
visceral             0
001_Glucose          0
002_BUN              0
003_Creatinine       0
UricAcid             0
005_Cholesterol      0
006_Triglyceride     0
008_HDL-C            0
009_LDL              0
010_AST              0
012_ALT              0
Alkaline_Phos        0
071_HbA1c           24
dtype: int64

In [194]:
dt10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,NaN,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,NaN,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation

### MEAN

In [195]:
from sklearn.impute import SimpleImputer

In [196]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
data_MEAN10 = mean_imputer.fit_transform(dt10.copy())
t_end = time.time()
t_MEAN10 = (t_end- t_start)

In [197]:
data_MEAN10 = pd.DataFrame(data_MEAN10)
data_MEAN10 = data_MEAN10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MEAN10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.600000,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.500000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,75.5,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.0,23.046923,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [198]:
mse_MEAN10 = mean_squared_error(data1,data_MEAN10)
mse_MEAN10

0.2726584462648351

### MISS Forest

In [199]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
data_MF10 = MF.fit_transform(dt10.copy())
t_end = time.time()
t_MF10= (t_end- t_start)

In [201]:
data_MF10 = pd.DataFrame(data_MF10)

In [202]:
data_MF10 = data_MF10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MF10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.00,19.300,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.00,21.300,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.00,21.600,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00,22.500,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.00,29.200,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,77.61,30.500,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00,27.427,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.00,22.900,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00,23.400,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [203]:
mse_MF10 = mean_squared_error(data1,data_MF10)
mse_MF10

0.1853980922574158

In [204]:
data_MF10.isna().sum()

Sex                        0
AgeSYear                   0
Weight                     0
Height                     0
BMI                        0
Systolic                   0
Diastolic                  0
HeartRate                  0
SMM                        0
Fat Mass                   0
% Body fat                 0
waist to hip radio         0
abdominal circumference    0
visceral fat area          0
Glucose                    0
BUN                        0
Creatinine                 0
Uric Acid                  0
Cholesterol                0
Triglyceride               0
HDL-C                      0
LDL                        0
AST                        0
ALT                        0
Alkaline Phos              0
HbA1c                      0
dtype: int64

### KNN

In [205]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [206]:
t_start = time.time()
KNN_imputer = KNNImputer(n_neighbors = 15)
data_KNN10 = KNN_imputer.fit_transform(dt10.copy())
t_end = time.time()
t_KNN10= (t_end- t_start)

In [207]:
data_KNN10 = pd.DataFrame(data_KNN10)

In [208]:
data_KNN10 = data_KNN10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_KNN10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.600000,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.500000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,73.4,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.0,24.213333,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [308]:
mse_KNN10 = mean_squared_error(data1,data_KNN10)
mse_KNN10 

0.2839703703703704

### MICE

In [210]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [211]:
t_start = time.time()
lr = LinearRegression()
imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_LR10 = imp.fit_transform(dt10.copy())
t_end = time.time()
t_LR10 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.11
[IterativeImputer] Change: 12.155915594238337, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.18
[IterativeImputer] Change: 3.2087633848436212, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.23
[IterativeImputer] Change: 0.19475364291383812, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [212]:
data_LR10 = pd.DataFrame(data_LR10)

In [213]:
data_LR10 = data_LR10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_LR10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.00000,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.00000,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.00000,21.600000,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00000,22.500000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.00000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,70.65223,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00000,27.555818,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.00000,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [307]:
mse_LR10 = mean_squared_error(data1,data_LR10)
mse_LR10 

0.18878682021825158

#### Support Vector Machine model

In [215]:
t_start = time.time()
regr = svm.SVR()
svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_SVM10 = svm_svr.fit_transform(dt10.copy())
t_end = time.time()
t_SVM10 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.09
[IterativeImputer] Change: 2.5929878542366875, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.24
[IterativeImputer] Change: 0.007715229668264101, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [216]:
data_SVM10 = pd.DataFrame(data_SVM10)

In [217]:
data_SVM10 = data_SVM10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_SVM10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.3000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.3000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,21.6000,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.5000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.2000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,74.142478,30.5000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,23.1704,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.9000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.4000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [218]:
mse_SVM10 = mean_squared_error(data1,data_SVM10)
mse_SVM10 

0.2453936252761394

### EM

In [219]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [220]:
t_start = time.time()
data_EM10 = impy.em(dt10.values.copy())
t_end = time.time()
t_EM10 = (t_end- t_start)

In [221]:
data_EM10 = pd.DataFrame(data_EM10)

In [222]:
data_EM10 = data_EM10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_EM10

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,21.600000,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.500000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,64.360953,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,22.338412,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [223]:
mse_EM10 = mean_squared_error(data1,data_EM10)
mse_EM10 

0.5107616098019754

# 20 %

## แทนค่า NaN

In [ ]:
random_column = np.random.choice(list1,int((153*17)*(20/100)),p=list_mis)
random_column = list(random_column)
random_column

In [225]:
weight = []
height = []
bmi = []
systolic = []
diastolic = []
heartrate = []
smm = []
fatmass = []
bodyfat = []
wait = []
abdominal = []
visceral = []
glucose = []
uric_acid = []
ALT = []
Alkaline = []
HbA1c = []

In [226]:
for i in random_column:
  if i == 'Weight':
    weight.append(i)  
  elif i == 'Height':
    height.append(i)
  elif i == 'BMI':
    bmi.append(i)
  elif i == 'Systolic':
    systolic.append(i)
  elif i == 'Diastolic':
    diastolic.append(i)
  elif i == 'HeartRate':
    heartrate.append(i)
  elif i == 'SMM':
    smm.append(i)
  elif i == 'Fat Mass':
    fatmass.append(i)
  elif i == '% Body fat':
    bodyfat.append(i)
  elif i == 'waist to hip radio':
    wait.append(i)
  elif i == 'abdominal circumference':
    abdominal.append(i)
  elif i == 'visceral fat area':
    visceral.append(i)
  elif i == '001_Glucose':
    glucose.append(i)
  elif i == '004_Uric Acid':
    uric_acid.append(i)
  elif i == '012_ALT':
    ALT.append(i)
  elif i == '013_Alkaline Phos':
    Alkaline.append(i)
  elif i == '071_HbA1c':
    HbA1c.append(i)

In [227]:
list2 = [len(weight),len(height),len(bmi),len(systolic),len(diastolic),len(heartrate),len(smm),
         len(fatmass),len(bodyfat),len(wait),len(abdominal),len(visceral),len(glucose),len(uric_acid),
         len(ALT),len(Alkaline),len(HbA1c)]
list3 = []
list4 = []

for i in range(153):
  m = 1/153
  list3.append(i)
  list4.append(m)

In [228]:
list2

[0, 2, 4, 1, 1, 16, 72, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50]

In [229]:
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [ ]:
list_071_HbA1c

In [231]:
prob_Weight

0.0

In [232]:
dt20 = data1.copy()
dt20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [233]:
for i,j in zip(list1,list2):
  dt20[i].ravel()[np.random.choice(list3,j,p= list4, replace=False)] = np.nan

In [234]:
dt20.isna().sum()

Sex                  0
AgeSYear             0
Weight               0
Height               2
BMI                  4
Systolic             1
Diastolic            1
HeartRate           16
SMM                 72
FatMass              0
Bodyfat              0
waisttohip           0
abdominal            0
visceral             0
001_Glucose          0
002_BUN              0
003_Creatinine       0
UricAcid             0
005_Cholesterol      0
006_Triglyceride     0
008_HDL-C            0
009_LDL              0
010_AST              0
012_ALT              0
Alkaline_Phos        0
071_HbA1c           50
dtype: int64

In [235]:
dt20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,NaN,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,NaN,NaN,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,NaN,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,NaN
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,NaN,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,NaN,NaN,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


## Imputation

### MEAN

In [236]:
from sklearn.impute import SimpleImputer

In [237]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
data_MEAN20 = mean_imputer.fit_transform(dt20.copy())
t_end = time.time()
t_MEAN20 = (t_end- t_start)

In [238]:
data_MEAN20 = pd.DataFrame(data_MEAN20)
data_MEAN20 = data_MEAN20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MEAN20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,75.79562,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,75.79562,22.507407,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.00000,22.507407,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.461165
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00000,22.507407,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.00000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.00000,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00000,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,75.79562,22.507407,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.400000


In [239]:
mse_MEAN20 = mean_squared_error(data1,data_MEAN20)
mse_MEAN20

0.7415277586948711

### MISS Forest

In [240]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
data_MF20 = MF.fit_transform(dt20.copy())
t_end = time.time()
t_MF20 = (t_end- t_start)

In [242]:
data_MF20 = pd.DataFrame(data_MF20)

In [243]:
data_MF20 = data_MF20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MF20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,74.32,19.300,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,76.64,20.846,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.00,22.417,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.811
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00,22.750,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.00,29.200,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.00,30.500,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00,27.800,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,73.47,23.037,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00,23.400,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.400


In [244]:
mse_MF20 = mean_squared_error(data1,data_MF20)
mse_MF20

0.40159790698843617

In [245]:
data_MF20.isna().sum()

Sex                        0
AgeSYear                   0
Weight                     0
Height                     0
BMI                        0
Systolic                   0
Diastolic                  0
HeartRate                  0
SMM                        0
Fat Mass                   0
% Body fat                 0
waist to hip radio         0
abdominal circumference    0
visceral fat area          0
Glucose                    0
BUN                        0
Creatinine                 0
Uric Acid                  0
Cholesterol                0
Triglyceride               0
HDL-C                      0
LDL                        0
AST                        0
ALT                        0
Alkaline Phos              0
HbA1c                      0
dtype: int64

### KNN

In [246]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [247]:
t_start = time.time()
KNN_imputer = KNNImputer(n_neighbors = 15)
data_KNN20 = KNN_imputer.fit_transform(dt20.copy())
t_end = time.time()
t_KNN20= (t_end- t_start)

In [248]:
data_KNN20 = pd.DataFrame(data_KNN20)

In [249]:
data_KNN20 = data_KNN20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_KNN20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,74.000000,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,75.866667,22.073333,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,21.306667,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.466667
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,21.280000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,73.600000,21.960000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.400000


In [250]:
mse_KNN20 = mean_squared_error(data1,data_KNN20)
mse_KNN20 

0.7196991006089045

### MICE

In [251]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_LR20 = imp.fit_transform(dt20.copy())
t_end = time.time()
t_LR20 = (t_end- t_start)

In [253]:
data_LR20 = pd.DataFrame(data_LR20)

In [254]:
data_LR20 = data_LR20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_LR20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,74.853503,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.377185,21.221535,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,20.684155,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,9.845381
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,22.180785,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,67.505455,22.909204,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.400000


In [255]:
mse_LR20 = mean_squared_error(data1,data_LR20)
mse_LR20 

0.751868446661129

#### Support Vector Machine model

In [256]:
t_start = time.time()
regr = svm.SVR()
svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_SVM20 = svm_svr.fit_transform(dt20.copy())
t_end = time.time()
t_SVM20 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.12
[IterativeImputer] Change: 3.4240228347869834, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.23
[IterativeImputer] Change: 0.009529793527322639, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [257]:
data_SVM20 = pd.DataFrame(data_SVM20)

In [258]:
data_SVM20 = data_SVM20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_SVM20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,74.936424,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,75.121348,21.330972,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,20.369371,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.522099
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,21.141347,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,74.609673,20.957113,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.400000


In [259]:
mse_SVM20 = mean_squared_error(data1,data_SVM20)
mse_SVM20 

0.7003123296320175

### EM

In [260]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [261]:
t_start = time.time()
data_EM20 = impy.em(dt20.values.copy())
t_end = time.time()
t_EM20 = (t_end- t_start)

In [262]:
data_EM20 = pd.DataFrame(data_EM20)

In [263]:
data_EM20 = data_EM20.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_EM20

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,43.647372,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,81.007864,17.907932,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,20.228178,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,3.984903
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,20.237347,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.000000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.000000,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,78.079037,19.222242,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.400000


In [264]:
mse_EM20 = mean_squared_error(data1,data_EM20)
mse_EM20 

1.7123871668854134

# 30 %

## แทนค่า NaN

In [ ]:
random_column = np.random.choice(list1,int((153*17)*(30/100)),p=list_mis)
random_column = list(random_column)
random_column

In [266]:
weight = []
height = []
bmi = []
systolic = []
diastolic = []
heartrate = []
smm = []
fatmass = []
bodyfat = []
wait = []
abdominal = []
visceral = []
glucose = []
uric_acid = []
ALT = []
Alkaline = []
HbA1c = []

In [267]:
for i in random_column:
  if i == 'Weight':
    weight.append(i)  
  elif i == 'Height':
    height.append(i)
  elif i == 'BMI':
    bmi.append(i)
  elif i == 'Systolic':
    systolic.append(i)
  elif i == 'Diastolic':
    diastolic.append(i)
  elif i == 'HeartRate':
    heartrate.append(i)
  elif i == 'SMM':
    smm.append(i)
  elif i == 'Fat Mass':
    fatmass.append(i)
  elif i == '% Body fat':
    bodyfat.append(i)
  elif i == 'waist to hip radio':
    wait.append(i)
  elif i == 'abdominal circumference':
    abdominal.append(i)
  elif i == 'visceral fat area':
    visceral.append(i)
  elif i == '001_Glucose':
    glucose.append(i)
  elif i == '004_Uric Acid':
    uric_acid.append(i)
  elif i == '012_ALT':
    ALT.append(i)
  elif i == '013_Alkaline Phos':
    Alkaline.append(i)
  elif i == '071_HbA1c':
    HbA1c.append(i)

In [268]:
list2 = [len(weight),len(height),len(bmi),len(systolic),len(diastolic),len(heartrate),len(smm),
         len(fatmass),len(bodyfat),len(wait),len(abdominal),len(visceral),len(glucose),len(uric_acid),
         len(ALT),len(Alkaline),len(HbA1c)]
list3 = []
list4 = []

for i in range(153):
  m = 1/153
  list3.append(i)
  list4.append(m)

In [269]:
list2

[3, 3, 5, 2, 4, 18, 83, 0, 0, 0, 0, 0, 0, 0, 1, 0, 83]

In [270]:
list1

['Weight',
 'Height',
 'BMI',
 'Systolic',
 'Diastolic',
 'HeartRate',
 'SMM',
 'FatMass',
 'Bodyfat',
 'waisttohip',
 'abdominal',
 'visceral',
 '001_Glucose',
 'UricAcid',
 '012_ALT',
 'Alkaline_Phos',
 '071_HbA1c']

In [ ]:
list_071_HbA1c

In [272]:
prob_Weight

0.0

In [273]:
dt30 = data1.copy()
dt30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.5,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,30.5,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,72.0,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.4,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [274]:
for i,j in zip(list1,list2):
  dt30[i].ravel()[np.random.choice(list3,j,p= list4, replace=False)] = np.nan

In [275]:
dt30.isna().sum()

Sex                  0
AgeSYear             0
Weight               3
Height               3
BMI                  5
Systolic             2
Diastolic            4
HeartRate           18
SMM                 83
FatMass              0
Bodyfat              0
waisttohip           0
abdominal            0
visceral             0
001_Glucose          0
002_BUN              0
003_Creatinine       0
UricAcid             0
005_Cholesterol      0
006_Triglyceride     0
008_HDL-C            0
009_LDL              0
010_AST              0
012_ALT              1
Alkaline_Phos        0
071_HbA1c           83
dtype: int64

In [276]:
dt30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,FatMass,...,003_Creatinine,UricAcid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,Alkaline_Phos,071_HbA1c
0,2,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,NaN
2,2,62.0,54.4,165.0,20.0,95.0,75.0,83.0,NaN,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,56.3,161.0,21.7,95.0,57.0,72.0,NaN,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,72.3,170.0,25.0,133.0,77.0,NaN,29.2,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,72.0,170.0,24.9,131.0,74.0,69.0,NaN,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,62.0,168.0,22.0,103.0,76.0,NaN,27.8,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.9,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,61.8,159.0,24.4,117.0,60.0,72.0,NaN,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,NaN


## Imputation

### MEAN

In [277]:
from sklearn.impute import SimpleImputer

In [278]:
t_start = time.time()
mean_imputer = SimpleImputer(strategy='mean')
data_MEAN30 = mean_imputer.fit_transform(dt30.copy())
t_end = time.time()
t_MEAN30 = (t_end- t_start)

In [279]:
data_MEAN30 = pd.DataFrame(data_MEAN30)
data_MEAN30 = data_MEAN10.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MEAN30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.0,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.0,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.0,21.600000,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.0,22.500000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.0,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,75.5,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.0,23.046923,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.0,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.0,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [280]:
mse_MEAN30 = mean_squared_error(data1,data_MEAN30)
mse_MEAN30

0.2726584462648351

### MISS Forest

In [281]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
t_start = time.time()
MF = MissForest()
data_MF30 = MF.fit_transform(dt30.copy())
t_end = time.time()
t_MF30= (t_end- t_start)

In [283]:
data_MF30 = pd.DataFrame(data_MF30)

In [284]:
data_MF30 = data_MF30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_MF30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.00,19.300,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.00,21.300,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,4.664
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.00,21.849,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00,22.375,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,72.24,29.200,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.00,29.159,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,71.69,27.800,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.00,22.900,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00,21.456,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.444


In [285]:
mse_MF30 = mean_squared_error(data1,data_MF30)
mse_MF30

0.7087306767219708

In [286]:
data_MF30.isna().sum()

Sex                        0
AgeSYear                   0
Weight                     0
Height                     0
BMI                        0
Systolic                   0
Diastolic                  0
HeartRate                  0
SMM                        0
Fat Mass                   0
% Body fat                 0
waist to hip radio         0
abdominal circumference    0
visceral fat area          0
Glucose                    0
BUN                        0
Creatinine                 0
Uric Acid                  0
Cholesterol                0
Triglyceride               0
HDL-C                      0
LDL                        0
AST                        0
ALT                        0
Alkaline Phos              0
HbA1c                      0
dtype: int64

### KNN

In [287]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [288]:
t_start = time.time()
KNN_imputer = KNNImputer(n_neighbors = 15)
data_KNN30 = KNN_imputer.fit_transform(dt30.copy())
t_end = time.time()
t_KNN30= (t_end- t_start)

In [289]:
data_KNN30 = pd.DataFrame(data_KNN30)

In [290]:
data_KNN30 = data_KNN30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_KNN30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,4.966667
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,20.833333,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,21.446667,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,75.933333,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,23.753333,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,76.133333,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,24.440000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.360000


In [291]:
mse_KNN30 = mean_squared_error(data1,data_KNN30)
mse_KNN30 

1.1964123233338915

### MICE

In [292]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

#### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_LR30 = imp.fit_transform(dt30.copy())
t_end = time.time()
t_LR30 = (t_end- t_start)

In [294]:
data_LR30 = pd.DataFrame(data_LR30)

In [295]:
data_LR30 = data_LR30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_LR30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,4.345878
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,22.234441,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,21.143972,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,77.925563,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,30.445063,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,36.263861,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,23.836198,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.535748


In [296]:
mse_LR30 = mean_squared_error(data1,data_LR10)
mse_LR30 

0.18878682021825158

#### Support Vector Machine model

In [297]:
t_start = time.time()
regr = svm.SVR()
svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data_SVM30 = svm_svr.fit_transform(dt30.copy())
t_end = time.time()
t_SVM30 = (t_end- t_start)

[IterativeImputer] Completing matrix with shape (153, 26)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.08
[IterativeImputer] Change: 3.86594244119574, scaled tolerance: 0.274 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.15
[IterativeImputer] Change: 0.01330226414343727, scaled tolerance: 0.274 
[IterativeImputer] Early stopping criterion reached.


In [298]:
data_SVM30 = pd.DataFrame(data_SVM30)

In [299]:
data_SVM30 = data_SVM30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_SVM30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.000000,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.000000,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.252343
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.000000,21.016270,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.000000,21.526580,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.100000
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,73.986867,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,69.000000,23.088830,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.500000
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,74.088454,27.800000,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.800000
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.000000,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.800000
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.000000,22.195581,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.617049


In [300]:
mse_SVM30 = mean_squared_error(data1,data_SVM30)
mse_SVM30 

1.2507155745810659

### EM

In [301]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [302]:
t_start = time.time()
data_EM30 = impy.em(dt10.values.copy())
t_end = time.time()
t_EM30 = (t_end- t_start)

In [303]:
data_EM30 = pd.DataFrame(data_EM30)

In [304]:
data_EM30 = data_EM30.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})
data_EM30

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,57.0,47.4,153.0,20.2,114.0,75.0,69.00000,19.300000,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2.0,24.0,53.2,157.0,21.6,103.0,69.0,78.00000,21.300000,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2.0,62.0,54.4,165.0,20.0,95.0,75.0,83.00000,21.600000,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2.0,35.0,56.3,161.0,21.7,95.0,57.0,72.00000,22.500000,15.4,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1.0,46.0,72.3,170.0,25.0,133.0,77.0,107.00000,29.200000,19.0,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1.0,60.0,72.0,170.0,24.9,131.0,74.0,58.73771,30.500000,13.8,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1.0,58.0,62.0,168.0,22.0,103.0,76.0,72.00000,22.181416,11.9,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2.0,42.0,55.5,165.0,20.4,102.0,52.0,77.00000,22.900000,14.1,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2.0,61.0,61.8,159.0,24.4,117.0,60.0,72.00000,23.400000,19.2,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [305]:
mse_EM30 = mean_squared_error(data1,data_EM30)
mse_EM30 

0.386506487145612

# MSE

In [309]:
mse_value = {'Method':['MEAN','Miss Forest','KNN','MICE-LR','MICE-SVM','EM'],'10%':[mse_MEAN10 ,mse_MF10 ,mse_KNN10 ,mse_LR10 ,mse_SVM10 ,mse_EM10 ],
      '20%':[mse_MEAN20 ,mse_MF20 ,mse_KNN20 ,mse_LR20 ,mse_SVM20,mse_EM20 ],'30%':[mse_MEAN30 ,mse_MF30 ,mse_KNN30 ,mse_LR30 ,mse_SVM30 ,mse_EM30 ]}

In [310]:
data_MSE = pd.DataFrame.from_dict(mse_value)
data_MSE

,Method,10%,20%,30%
0,MEAN,0.272658,0.741528,0.272658
1,Miss Forest,0.185398,0.401598,0.708731
2,KNN,0.283970,0.719699,1.196412
3,MICE-LR,0.188787,0.751868,0.188787
4,MICE-SVM,0.245394,0.700312,1.250716
5,EM,0.510762,1.712387,0.386506
